In [1]:
import argparse
import numpy as np
import pandas as pd
import torch
from tabulate import tabulate
from torch.utils.data import DataLoader

from model import BertCustomBinaryClassifier
from utils.ensemble_utils import make_predictions
from utils.evaluate_metrics import evaluate_metrics
from utils.data_preprocessing import load_dataset

In [2]:
import logging
logging.getLogger("transforkmer_values.modeling_utils").setLevel(logging.ERROR)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", type=int, default=64, help="")
parser.add_argument("--max_length", type=int, default=200, help="")
args = parser.parse_args(args=[])

# **BERT Models**

In [5]:
threshold = 0.50
kmer_values = [3, 4, 5, 6]
model_date = "2025-02-27_V2"

results = []  # List to store results
train_predictions_list, test_predictions_list = [], []  # Lists for storing model predictions
train_labels_list, test_labels_list = [], []  # Lists for storing true labels

print(f"Threshold: {threshold}")
print(f"Classifier model date: {model_date}")

for kmer in kmer_values:

    args.model_path = f"./outputs/classifier_models/{model_date}/{kmer}-mer"
    args.test_data_path = f"./data/enhancer_classification/{kmer}-mer_classification_test.txt"
    args.train_data_path = f"./data/enhancer_classification/{kmer}-mer_classification_train.txt"

    # Load training and test datasets
    train_dataset = load_dataset(args, validation=False)
    test_dataset = load_dataset(args, validation=True)

    # Initialize data loaders for batch processing
    train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)

    # Model
    model = BertCustomBinaryClassifier.from_pretrained(args.model_path, num_labels=1).to(device)

    # Prediction on training datasets
    train_predictions, train_labels = make_predictions(model, train_dataloader, kmer=kmer)
    train_predictions_list.append(train_predictions)
    train_labels_list.append(train_labels)

    acc, sn, sp, mcc, auc = evaluate_metrics(train_predictions, train_labels)
    results.append({"k-mer": kmer, "Dataset": "Train", "Accuracy": acc, "Sensitivity": sn, "Specificity": sp, "MCC": mcc, "AUC": auc})

    # Prediction on test (independent) dataset
    test_predictions, test_labels = make_predictions(model, test_dataloader, kmer=kmer)
    test_predictions_list.append(test_predictions)
    test_labels_list.append(test_labels)

    acc, sn, sp, mcc, auc = evaluate_metrics(test_predictions, test_labels)
    results.append({"k-mer": kmer, "Dataset": "Test", "Accuracy": acc, "Sensitivity": sn, "Specificity": sp, "MCC": mcc, "AUC": auc})

Threshold: 0.5
Classifier model date: 2025-02-27_V2


In [6]:
results_df = pd.DataFrame(results, columns=["k-mer", "Dataset", "Accuracy", "Sensitivity", "Specificity", "MCC", "AUC"])

# Split into training and test
training_df = results_df[results_df['Dataset'].str.contains("Train")]
test_df = results_df[results_df['Dataset'].str.contains("Test")]

print("Training results:")
print(tabulate(training_df, headers="keys", tablefmt="grid", showindex=False, floatfmt=".4f"))

print("\nTest results:")
print(tabulate(test_df, headers="keys", tablefmt="grid", showindex=False, floatfmt=".4f"))

Training results:
+---------+-----------+------------+---------------+---------------+--------+--------+
|   k-mer | Dataset   |   Accuracy |   Sensitivity |   Specificity |    MCC |    AUC |
+=========+===========+============+===============+===============+========+========+
|       3 | Train     |     0.8888 |        0.8302 |        0.9474 | 0.7830 | 0.9162 |
+---------+-----------+------------+---------------+---------------+--------+--------+
|       4 | Train     |     0.9111 |        0.8639 |        0.9582 | 0.8258 | 0.9218 |
+---------+-----------+------------+---------------+---------------+--------+--------+
|       5 | Train     |     0.8982 |        0.8329 |        0.9636 | 0.8034 | 0.8944 |
+---------+-----------+------------+---------------+---------------+--------+--------+
|       6 | Train     |     0.9043 |        0.8423 |        0.9663 | 0.8149 | 0.8987 |
+---------+-----------+------------+---------------+---------------+--------+--------+

Test results:
+---------

In [7]:
# Initialize variables
best_test_acc = 0.0
best_threshold = 0.0

# Weights for ensemble 
weights = np.array(results_df[results_df['Dataset'].str.contains("Train")]['Accuracy'])

# Normalize weights to ensure they sum to 1
weights /= np.sum(weights)

# Define threshold values to test
threshold_values = np.arange(0.40, 0.81, 0.001)

# Variables to store the best metrics
best_train_acc, best_train_sn, best_train_sp, best_train_mcc, best_train_auc = 0, 0, 0, 0, 0
best_test_acc, best_test_sn, best_test_sp, best_test_mcc, best_test_auc = 0, 0, 0, 0, 0

# Loop through threshold values
for threshold in threshold_values:
    # Weighted average for training predictions
    train_predictions_weighted = np.average(train_predictions_list, axis=0, weights=weights)
    train_labels_average = np.array(train_labels_list).mean(axis=0)
    
    # Evaluate training metrics
    train_acc, train_sn, train_sp, train_mcc, train_auc = evaluate_metrics(
        train_predictions_weighted, train_labels_average, threshold=threshold
    )

    # Weighted average for test predictions
    test_predictions_weighted = np.average(test_predictions_list, axis=0, weights=weights)
    test_labels_average = np.array(test_labels_list).mean(axis=0)

    # Evaluate test metrics
    test_acc, test_sn, test_sp, test_mcc, test_auc = evaluate_metrics(
        test_predictions_weighted, test_labels_average, threshold=threshold
    )
    
    # Check if current threshold yields the best test accuracy or compare MCC when accuracies are equal
    if (test_acc > best_test_acc) or (test_acc == best_test_acc and train_acc > best_train_acc):
        best_test_acc = test_acc
        best_threshold = threshold

        # Store the best metrics
        best_train_acc, best_train_sn, best_train_sp, best_train_mcc, best_train_auc = train_acc, train_sn, train_sp, train_mcc, train_auc
        best_test_acc, best_test_sn, best_test_sp, best_test_mcc, best_test_auc = test_acc, test_sn, test_sp, test_mcc, test_auc

# Print the best threshold and corresponding test accuracy
print(f"Best Threshold: {best_threshold:.4f}")
print(f"Best Test Accuracy: {best_test_acc:.4f}")

# Update results table with the best weighted ensemble metrics
ensemble_results_weighted = [
    ["Weighted Ensemble Training", f"{best_train_acc:.4f}", f"{best_train_sn:.4f}", f"{best_train_sp:.4f}", f"{best_train_mcc:.4f}", f"{best_train_auc:.4f}"],
    ["Weighted Ensemble Testing", f"{best_test_acc:.4f}", f"{best_test_sn:.4f}", f"{best_test_sp:.4f}", f"{best_test_mcc:.4f}", f"{best_test_auc:.4f}"]
]
ensemble_results_df_weighted = pd.DataFrame(ensemble_results_weighted, columns=["Dataset", "Accuracy", "Sensitivity", "Specificity", "MCC", "AUC"])

# Display results
print(tabulate(ensemble_results_df_weighted, headers="keys", tablefmt="grid", showindex=False, floatfmt=".4f"))
threshold = best_threshold


Best Threshold: 0.4930
Best Test Accuracy: 0.8850
+----------------------------+------------+---------------+---------------+--------+--------+
| Dataset                    |   Accuracy |   Sensitivity |   Specificity |    MCC |    AUC |
+============================+============+===============+===============+========+========+
| Weighted Ensemble Training |     0.9245 |        0.8706 |        0.9784 | 0.8540 | 0.9701 |
+----------------------------+------------+---------------+---------------+--------+--------+
| Weighted Ensemble Testing  |     0.8850 |        0.8600 |        0.9100 | 0.7710 | 0.9489 |
+----------------------------+------------+---------------+---------------+--------+--------+


In [8]:
# Calculate individual model scores for each sample
individual_model_scores = []
for i, kmer in enumerate(kmer_values):
    for sample_idx in range(len(test_predictions_list[i])):
        individual_model_scores.append({
            "Sample": sample_idx + 1,
            "k-mer": kmer,
            "Score": test_predictions_list[i][sample_idx]
        })

# Calculate ensemble model scores
ensemble_test_predictions = np.array(test_predictions_list).mean(axis=0)
for sample_idx in range(len(ensemble_test_predictions)):
    individual_model_scores.append({
        "Sample": sample_idx + 1,
        "k-mer": "Ensemble",
        "Score": ensemble_test_predictions[sample_idx]
    })

# Convert to DataFrame
individual_model_scores_df = pd.DataFrame(individual_model_scores)

# Reshape DataFrame to have each sample as a row, and each model as a column
pivot_df = individual_model_scores_df.pivot(index="Sample", columns="k-mer", values="Score")
pivot_df.reset_index(inplace=True)

# Adding 'Label' and 'Prediction' columns
pivot_df["Label"] = test_labels_list[0]  
pivot_df["Prediction"] = (pivot_df["Ensemble"] >= threshold).astype(int)  # Converting ensemble scores to binary predictions

# Rename columns to match desired format
# Assuming you have 4 k-mer values in kmer_values list
column_mapping = {kmer: str(i + 3) for i, kmer in enumerate(kmer_values)}
pivot_df.rename(columns=column_mapping, inplace=True)

# Add 'Correct' column
pivot_df["Correct"] = pivot_df["Label"] == pivot_df["Prediction"]

# Display the table
print(tabulate(pivot_df, headers="keys", tablefmt="grid", showindex=False, floatfmt=".4f"))

+----------+--------+--------+--------+--------+------------+---------+--------------+-----------+
|   Sample |      3 |      4 |      5 |      6 |   Ensemble |   Label |   Prediction | Correct   |
+==========+========+========+========+========+============+=========+==============+===========+
|        1 | 0.1364 | 0.0613 | 0.0750 | 0.2051 |     0.1195 |       0 |            0 | True      |
+----------+--------+--------+--------+--------+------------+---------+--------------+-----------+
|        2 | 0.1364 | 0.0613 | 0.0750 | 0.2051 |     0.1195 |       0 |            0 | True      |
+----------+--------+--------+--------+--------+------------+---------+--------------+-----------+
|        3 | 0.1365 | 0.0613 | 0.0750 | 0.2051 |     0.1195 |       0 |            0 | True      |
+----------+--------+--------+--------+--------+------------+---------+--------------+-----------+
|        4 | 0.1364 | 0.0613 | 0.2417 | 0.7400 |     0.2949 |       0 |            0 | True      |
+---------

In [9]:
threshold = round(threshold, 4)
print(f"Threshold: {threshold}\n")

Threshold: 0.493



In [10]:
# Calculate counts of correct and incorrect predictions for each label
label_counts = pivot_df.groupby(["Label", "Correct"]).size().unstack(fill_value=0)

# Display the counts
print(label_counts)

# Calculate incorrect predictions for each k-mer model
incorrect_counts = pivot_df[["Label"]].copy()
for kmer, col_name in column_mapping.items():
    incorrect_counts[col_name] = pivot_df["Label"] != (pivot_df[col_name] >= threshold).astype(int)

# Split DataFrame into two separate DataFrames based on labels
label_0_df = pivot_df[pivot_df["Label"] == 0]
label_1_df = pivot_df[pivot_df["Label"] == 1]

# Calculate incorrect predictions for each k-mer model for both labels separately
incorrect_counts_label_0 = label_0_df[["Label"]].copy()
incorrect_counts_label_1 = label_1_df[["Label"]].copy()

for kmer, col_name in column_mapping.items():
    incorrect_counts_label_0[col_name] = label_0_df["Label"] != (label_0_df[col_name] >= threshold).astype(int)
    incorrect_counts_label_1[col_name] = label_1_df["Label"] != (label_1_df[col_name] >= threshold).astype(int)

# Sum incorrect predictions for each k-mer model for both labels separately
incorrect_counts_sum_label_0 = incorrect_counts_label_0.sum(axis=0)[1:]
incorrect_counts_sum_label_1 = incorrect_counts_label_1.sum(axis=0)[1:]

# Combine the incorrect predictions into a single DataFrame
combined_incorrect_counts_df = pd.DataFrame({
    "Model": list(incorrect_counts_sum_label_0.index),
    "Incorrect Predictions (Label 0)": incorrect_counts_sum_label_0.values,
    "Incorrect Predictions (Label 1)": incorrect_counts_sum_label_1.values
})

# Add a sum row at the end
sum_row = pd.DataFrame({
    "Model": ["Sum"],
    "Incorrect Predictions (Label 0)": [incorrect_counts_sum_label_0.sum()],
    "Incorrect Predictions (Label 1)": [incorrect_counts_sum_label_1.sum()]
})

combined_incorrect_counts_df = pd.concat([combined_incorrect_counts_df, sum_row], ignore_index=True)

# Add a sum column for the rows
combined_incorrect_counts_df["Sum"] = combined_incorrect_counts_df["Incorrect Predictions (Label 0)"] + combined_incorrect_counts_df["Incorrect Predictions (Label 1)"]

# Display the combined incorrect predictions counts
print("\nCombined incorrect predictions counts:")
print(tabulate(combined_incorrect_counts_df, headers="keys", tablefmt="grid", showindex=False))

Correct  False  True 
Label                
0           10     90
1           14     86

Combined incorrect predictions counts:
+---------+-----------------------------------+-----------------------------------+-------+
| Model   |   Incorrect Predictions (Label 0) |   Incorrect Predictions (Label 1) |   Sum |
+=========+===================================+===================================+=======+
| 3       |                                14 |                                16 |    30 |
+---------+-----------------------------------+-----------------------------------+-------+
| 4       |                                11 |                                12 |    23 |
+---------+-----------------------------------+-----------------------------------+-------+
| 5       |                                10 |                                16 |    26 |
+---------+-----------------------------------+-----------------------------------+-------+
| 6       |                                1

# **Machine Learning Models**

In [13]:
from sklearn.ensemble import  ExtraTreesClassifier

In [14]:
# Prepare meta-features for training and testing
train_meta_features = np.column_stack(train_predictions_list)
test_meta_features = np.column_stack(test_predictions_list)

train_labels = train_labels_list[0]     
test_labels = test_labels_list[0]

In [15]:
model = ExtraTreesClassifier(n_estimators=1000, random_state=42)

threshold = 0.9999   

# Train the Stacking Classifier on the training set
model.fit(train_meta_features, train_labels)

# Meta-model predictions on the validation set
train_predictions_et = model.predict_proba(train_meta_features)[:, 1]
test_predictions_et = model.predict_proba(test_meta_features)[:, 1]

train_acc, train_sn, train_sp, train_mcc, train_auc = evaluate_metrics(train_predictions_et, train_labels, threshold=threshold)
test_acc, test_sn, test_sp, test_mcc, test_auc = evaluate_metrics(test_predictions_et, test_labels, threshold=threshold)

# Display results in table using tabulate
results = [["Train", train_acc, train_sn, train_sp, train_mcc, train_auc], ["Test", test_acc, test_sn, test_sp, test_mcc, test_auc]]

headers = ["Dataset", "Accuracy", "Sensitivity", "Specificity", "MCC", "AUC"]

print(tabulate(results, headers=headers, tablefmt="grid", floatfmt=".4f"))

+-----------+------------+---------------+---------------+--------+--------+
| Dataset   |   Accuracy |   Sensitivity |   Specificity |    MCC |    AUC |
+===========+============+===============+===============+========+========+
| Train     |     1.0000 |        1.0000 |        1.0000 | 1.0000 | 1.0000 |
+-----------+------------+---------------+---------------+--------+--------+
| Test      |     0.9900 |        1.0000 |        0.9800 | 0.9802 | 0.9900 |
+-----------+------------+---------------+---------------+--------+--------+


# **K-Fold**

## **5-fold**

In [ ]:
from sklearn.model_selection import KFold

# Parameters for k-fold
k_folds = 5  # Number of folds
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
threshold = 0.5

# Initialize results storage
kfold_results = []

# Convert meta-features and labels to numpy arrays for k-fold processing
train_meta_features = np.array(train_meta_features)
train_labels = np.array(train_labels)

# Perform k-fold cross-validation
for fold, (train_idx, val_idx) in enumerate(kf.split(train_meta_features)):
    print(f"Processing Fold {fold + 1}/{k_folds}")

    # Split data into training and validation sets for this fold
    X_train, X_val = train_meta_features[train_idx], train_meta_features[val_idx]
    y_train, y_val = train_labels[train_idx], train_labels[val_idx]

    # Initialize and train Extra Trees model
    model = ExtraTreesClassifier(n_estimators=1000, random_state=42)
    model.fit(X_train, y_train)

    # Predictions on validation set
    val_predictions = model.predict_proba(X_val)[:, 1]

    # Evaluate metrics 
    val_acc, val_sn, val_sp, val_mcc, val_auc = evaluate_metrics(val_predictions, y_val, threshold=threshold)

    # Store results for this fold
    kfold_results.append({"Fold": fold + 1, "ACC": val_acc, "SN": val_sn, "SP": val_sp, "MCC": val_mcc, "AUC": val_auc})

# Display aggregated results across folds
print(f'\n{k_folds}-fold cross-validation results:')
headers = ["Fold", "ACC", "SN", "SP", "MCC", "AUC"]
results_table = [[result["Fold"], result["ACC"], result["SN"], result["SP"], result["MCC"], result["AUC"]] for result in kfold_results]
print(tabulate(results_table, headers=headers, tablefmt="grid", floatfmt=".4f"))

Processing Fold 1/5
Processing Fold 2/5
Processing Fold 3/5
Processing Fold 4/5
Processing Fold 5/5

5-fold cross-validation results:
+--------+--------+--------+--------+--------+--------+
|   Fold |    ACC |     SN |     SP |    MCC |    AUC |
+========+========+========+========+========+========+
|      1 | 0.9125 | 0.8720 | 0.9624 | 0.8298 | 0.9575 |
+--------+--------+--------+--------+--------+--------+
|      2 | 0.9663 | 0.9403 | 0.9877 | 0.9325 | 0.9925 |
+--------+--------+--------+--------+--------+--------+
|      3 | 0.9293 | 0.9220 | 0.9359 | 0.8582 | 0.9666 |
+--------+--------+--------+--------+--------+--------+
|      4 | 0.9360 | 0.8865 | 0.9808 | 0.8746 | 0.9702 |
+--------+--------+--------+--------+--------+--------+
|      5 | 0.9426 | 0.9506 | 0.9328 | 0.8840 | 0.9804 |
+--------+--------+--------+--------+--------+--------+


In [44]:
# Calculate and display average results
avg_acc = np.mean([result["ACC"] for result in kfold_results])
avg_sn = np.mean([result["SN"] for result in kfold_results])
avg_sp = np.mean([result["SP"] for result in kfold_results])
avg_mcc = np.mean([result["MCC"] for result in kfold_results])
avg_auc = np.mean([result["AUC"] for result in kfold_results])

print(f'Average {k_folds}-fold cross-validation results:')
avg_results_table = [["Mean", avg_acc, avg_sn, avg_sp, avg_mcc, avg_auc]]
print(tabulate(avg_results_table, headers=headers, tablefmt="grid", floatfmt=".4f"))

Average 5-fold cross-validation results:
+--------+--------+--------+--------+--------+--------+
| Fold   |    ACC |     SN |     SP |    MCC |    AUC |
+========+========+========+========+========+========+
| Mean   | 0.9373 | 0.9143 | 0.9599 | 0.8758 | 0.9734 |
+--------+--------+--------+--------+--------+--------+


## **10-fold**

In [ ]:
# Parameters for k-fold
k_folds = 10  # Number of folds
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
threshold = 0.5

# Initialize results storage
kfold_results = []

# Convert meta-features and labels to numpy arrays for k-fold processing
train_meta_features = np.array(train_meta_features)
train_labels = np.array(train_labels)

# Perform k-fold cross-validation
for fold, (train_idx, val_idx) in enumerate(kf.split(train_meta_features)):
    print(f"Processing Fold {fold + 1}/{k_folds}")

    # Split data into training and validation sets for this fold
    X_train, X_val = train_meta_features[train_idx], train_meta_features[val_idx]
    y_train, y_val = train_labels[train_idx], train_labels[val_idx]

    # Initialize and train Extra Trees model
    model = ExtraTreesClassifier(n_estimators=1000, random_state=42)
    model.fit(X_train, y_train)

    # Predictions on validation set
    val_predictions = model.predict_proba(X_val)[:, 1]

    # Evaluate metrics 
    val_acc, val_sn, val_sp, val_mcc, val_auc = evaluate_metrics(val_predictions, y_val, threshold=threshold)

    # Store results for this fold
    kfold_results.append({"Fold": fold + 1, "ACC": val_acc, "SN": val_sn, "SP": val_sp, "MCC": val_mcc, "AUC": val_auc})

# Display aggregated results across folds
print(f'\n{k_folds}-fold cross-validation results:')
headers = ["Fold", "ACC", "SN", "SP", "MCC", "AUC"]
results_table = [[result["Fold"], result["ACC"], result["SN"], result["SP"], result["MCC"], result["AUC"]] for result in kfold_results]
print(tabulate(results_table, headers=headers, tablefmt="grid", floatfmt=".4f"))

Processing Fold 1/10
Processing Fold 2/10
Processing Fold 3/10
Processing Fold 4/10
Processing Fold 5/10
Processing Fold 6/10
Processing Fold 7/10
Processing Fold 8/10
Processing Fold 9/10
Processing Fold 10/10

10-fold cross-validation results:
+--------+--------+--------+--------+--------+--------+
|   Fold |    ACC |     SN |     SP |    MCC |    AUC |
+========+========+========+========+========+========+
|      1 | 0.9396 | 0.9250 | 0.9565 | 0.8796 | 0.9671 |
+--------+--------+--------+--------+--------+--------+
|      2 | 0.8792 | 0.8214 | 0.9538 | 0.7694 | 0.9525 |
+--------+--------+--------+--------+--------+--------+
|      3 | 0.9732 | 0.9375 | 1.0000 | 0.9462 | 0.9858 |
+--------+--------+--------+--------+--------+--------+
|      4 | 0.9664 | 0.9583 | 0.9740 | 0.9329 | 0.9896 |
+--------+--------+--------+--------+--------+--------+
|      5 | 0.9257 | 0.8889 | 0.9529 | 0.8478 | 0.9595 |
+--------+--------+--------+--------+--------+--------+
|      6 | 0.9392 | 0.9605

In [46]:
# Calculate and display average results
avg_acc = np.mean([result["ACC"] for result in kfold_results])
avg_sn = np.mean([result["SN"] for result in kfold_results])
avg_sp = np.mean([result["SP"] for result in kfold_results])
avg_mcc = np.mean([result["MCC"] for result in kfold_results])
avg_auc = np.mean([result["AUC"] for result in kfold_results])

print(f'Average {k_folds}-fold cross-validation results:')
avg_results_table = [["Mean", avg_acc, avg_sn, avg_sp, avg_mcc, avg_auc]]
print(tabulate(avg_results_table, headers=headers, tablefmt="grid", floatfmt=".4f"))

Average 10-fold cross-validation results:
+--------+--------+--------+--------+--------+--------+
| Fold   |    ACC |     SN |     SP |    MCC |    AUC |
+========+========+========+========+========+========+
| Mean   | 0.9387 | 0.9171 | 0.9594 | 0.8787 | 0.9719 |
+--------+--------+--------+--------+--------+--------+
